<img src="images/logo.png" width="300">

In [1]:
#import modules

import numpy as np
import pandas as pd
import ipywidgets as widg
import IPython.display as idisp
import plotly.graph_objects as go


In [2]:
#color settings

predify_color = '0,71,187' #Predify dark blue

#print markdown text in color
def html( prefix, text ):
    idisp.display( idisp.Markdown( '{} <span style="color: rgb({})">{}</span>'.format( prefix, predify_color, text ) ) )
    

In [3]:
html( '#', 'Análisis de spots (Auditsa) y ratings (Inra) en la radio mexicana' )

# <span style="color: rgb(0,71,187)">Análisis de spots (Auditsa) y ratings (Inra) en la radio mexicana</span>

In [4]:
#initial settings

loadflag = 'y' #load files ('y') or calculate from scratch ('n')

dataloc = 'data/' #location of raw datafiles
imgloc = 'images/' #location of images

#plot settings

months = [ 'enero', 'febrero', 'marzo', 'abril', 'mayo' ] #months in right order

plot_props = { 'marker' : 'o',
    'ms' : 12,
    'ls' : '-',
    'lw' : 3,
    'fontsize' : 20,
    'xylabel' : 20,
    'ticklabel' : 15,
    'legend_prop' : { 'size':15 },
    'legend_hlen' : 2,
    'legend_np' : 2 }


In [5]:
#data loading

savename = dataloc+'data.h5' #HDF5 savename

data = {} #raw data
data['auditsa'] = pd.read_hdf( savename, key='auditsa' )
data['inra'] = pd.read_hdf( savename, key='inra' )
data['join'] = pd.read_hdf( savename, key='join' )


In [6]:
html( '##', 'Dashboard Auditsa' )

## <span style="color: rgb(0,71,187)">Dashboard Auditsa</span>

In [7]:
#plot total number of spots in stations

#define widgets

#station 1
est_1_widg = widg.Dropdown( options=data['auditsa']['estacion'].sort_values().unique(),
                            value='CDMX XHMVS-FM - (102.5 FM) MVS 102.5',
                            description='Estación 1:' )
#station 2
est_2_widg = widg.Dropdown( options=data['auditsa']['estacion'].sort_values().unique(),
                            value='CDMX XHEXA-FM - (104.9 FM) Exa',
                            description='Estación 2:' )
#filter
filtro_widg = widg.widgets.SelectMultiple( options=data['auditsa']['filtro'].sort_values().unique(),
                                           value=data['auditsa']['filtro'].sort_values().unique().tolist(),
                                           description='Filtro:' )
#sector
sector_widg = widg.Dropdown( options=data['auditsa']['sector'].sort_values().dropna().unique(),
                             value='COMPUTACION',
                             description='Sector:' )

#plot total number of spots in stations
def plot_spots_station( estacion_1, estacion_2, filtro, sector ):
    
    cond_1 = ( ( data['auditsa']['filtro'].isin( filtro ) )
             & ( data['auditsa']['sector'] == sector )
             & ( data['auditsa']['estacion'] == estacion_1 )
             ) #condition to filter data
    yplot_1 = data['auditsa'][ cond_1 ].groupby( 'mes' )['hits'].agg( np.sum ).reindex( months ) #aggregate by month

    cond_2 = ( ( data['auditsa']['filtro'].isin( filtro ) )
             & ( data['auditsa']['sector'] == sector )
             & ( data['auditsa']['estacion'] == estacion_2 )
             ) #condition to filter data
    yplot_2 = data['auditsa'][ cond_2 ].groupby( 'mes' )['hits'].agg( np.sum ).reindex( months ) #aggregate by month
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Número de spots en estaciones',
                       xaxis_title='mes',
                       yaxis_title='Número de spots' )
    
    #plot plot!
    fig.add_trace( go.Scatter( x=yplot_1.index, y=yplot_1, name=estacion_1 ) )
    fig.add_trace( go.Scatter( x=yplot_2.index, y=yplot_2, name=estacion_2 ) )
    
    #finalise plot
    fig.update_layout( hovermode='x' )
    fig.show()
    
#display interactive widget
idisp.display( widg.interactive( plot_spots_station, estacion_1=est_1_widg, estacion_2=est_2_widg,
                                 filtro=filtro_widg, sector=sector_widg ) )


interactive(children=(Dropdown(description='Estación 1:', index=49, options=('CDMX XEABC-AM - (760 AM) ABC Rad…

In [8]:
#plot company spot distribution among station groups

#define widgets

#company
comp_widg = widg.Dropdown( options=data['auditsa']['compania'].sort_values().unique(),
                           value='Coca-Cola',
                           #value='Cooperativa Pascual',
                           description='Compañía:' )
#month 1
mes_1_widg = widg.Dropdown( options=months,
                            value='marzo',
                            description='Mes 1:' )
#month 2
mes_2_widg = widg.Dropdown( options=months,
                            value='abril',
                            description='Mes 2:' )
    
#plot company spot distribution among station groups
def plot_spotDist_groups( compania, mes_1, mes_2 ):

    cond_1 = ( ( data['auditsa']['compania'] == compania )
             & ( data['auditsa']['mes'] == mes_1 )
             )
    yplot_1 = data['auditsa'][ cond_1 ].groupby( 'competencia' )['hits'].agg( np.sum ).sort_values( ascending=False ) #aggregate by group

    cond_2 = ( ( data['auditsa']['compania'] == compania )
             & ( data['auditsa']['mes'] == mes_2 )
             )
    yplot_2 = data['auditsa'][ cond_2 ].groupby( 'competencia' )['hits'].agg( np.sum ).sort_values( ascending=False ) #aggregate by group


    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Spots por grupo en compañía',
                       xaxis_title='Grupo',
                       yaxis_title='Spots' )

    #plot plot!
    fig.add_trace( go.Bar( x=yplot_1.index, y=yplot_1, name=mes_1 ) )
    fig.add_trace( go.Bar( x=yplot_2.index, y=yplot_2, name=mes_2 ) )
    
    #finalise plot
    fig.update_layout( barmode='group', xaxis_tickangle=-45, hovermode='x' )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_spotDist_groups, compania=comp_widg, mes_1=mes_1_widg, mes_2=mes_2_widg ) )


interactive(children=(Dropdown(description='Compañía:', index=672, options=('17 Asados', '3 Hermanos', '3A Res…

In [9]:
html( '##', 'Dashboard Inra' )

## <span style="color: rgb(0,71,187)">Dashboard Inra</span>

In [10]:
#plot audience in month for all stations

#define widgets

#station 1
mes_1_widg = widg.Dropdown( options=months,
                            value='marzo',
                            description='Mes 1:' )
#station 2
mes_2_widg = widg.Dropdown( options=months,
                            value='abril',
                            description='Mes 2:' )
#days
dias_widg = widg.Dropdown( options=data['inra']['dias'].sort_values().unique(),
                                value='L-V',
                                description='Días:' )
#place
plaza_widg = widg.SelectMultiple( options=data['inra']['plaza'].sort_values().unique(),
                            value=['Ciudad de México'],
                            description='Plaza:' )
#target
target_widg = widg.RadioButtons( options=data['inra']['target'].sort_values().unique(),
                                 value='Total Población',
                                 description='Target:' )

#plot audience in month for all stations
def plot_audience_month( mes_1, mes_2, dias, plaza, target ):

    cond_1 = ( ( data['inra']['mes'] == mes_1 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['plaza'].isin( plaza ) )
             & ( data['inra']['target'] == target )
             )
    yplot_1 = data['inra'][ cond_1 ].groupby( 'estacion' )['audiencia'].agg( np.sum ).sort_values( ascending=False ) #aggregate by station

    cond_2 = ( ( data['inra']['mes'] == mes_2 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['plaza'].isin( plaza ) )
             & ( data['inra']['target'] == target )
             )
    yplot_2 = data['inra'][ cond_2 ].groupby( 'estacion' )['audiencia'].agg( np.sum ).sort_values( ascending=False ) #aggregate by station

    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Audiencia por estación',
                       xaxis_title='Estación',
                       yaxis_title='Audiencia' )

    #plot plot!
    fig.add_trace( go.Bar( x=yplot_1.index, y=yplot_1, name=mes_1 ) )
    fig.add_trace( go.Bar( x=yplot_2.index, y=yplot_2, name=mes_2 ) )
    
    #finalise plot
    fig.update_layout( barmode='group', xaxis_tickangle=-45, hovermode='x' )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_audience_month, mes_1=mes_1_widg, mes_2=mes_2_widg, dias=dias_widg,
                                 plaza=plaza_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Mes 1:', index=2, options=('enero', 'febrero', 'marzo', 'abril', '…

In [11]:
#plot size of audience per hour

#define widgets

#station 1
est_1_widg = widg.Dropdown( options=data['inra']['estacion'].sort_values().unique(),
                            value='CDMX XHMVS-FM - (102.5 FM) MVS 102.5',
                            description='Estación 1:' )
#station 2
est_2_widg = widg.Dropdown( options=data['inra']['estacion'].sort_values().unique(),
                            value='CDMX XHEXA-FM - (104.9 FM) Exa',
                            description='Estación 2:' )
#days
dias_widg = widg.Dropdown( options=data['inra']['dias'].sort_values().unique(),
                                value='L-V',
                                description='Días:' )
#month
mes_widg = widg.SelectMultiple( options=months,
                            value=months,
                            description='Mes:' )
#target
target_widg = widg.RadioButtons( options=data['inra']['target'].sort_values().unique(),
                                 value='Total Población',
                                 description='Target:' )
   
#plot size of audience per hour
def plot_audience_hour( estacion_1, estacion_2, dias, mes, target ):
    
    cond_1 = ( ( data['inra']['estacion'] == estacion_1 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['mes'].isin( mes ) )
             & ( data['inra']['target'] == target )
             ) #condition to filter data
    yplot_1 = data['inra'][ cond_1 ].groupby( 'hora' )['audiencia'].agg( np.sum ).sort_index() #aggregate by hours

    cond_2 = ( ( data['inra']['estacion'] == estacion_2 )
             & ( data['inra']['dias'] == dias )
             & ( data['inra']['mes'].isin( mes ) )
             & ( data['inra']['target'] == target )
             ) #condition to filter data
    yplot_2 = data['inra'][ cond_2 ].groupby( 'hora' )['audiencia'].agg( np.sum ).sort_index() #aggregate by hours
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Audiencia por hora',
                       xaxis_title='Hora',
                       yaxis_title='Audiencia' )
    
    #plot plot!
    fig.add_trace( go.Scatter( x=yplot_1.index, y=yplot_1, name=estacion_1 ) )
    fig.add_trace( go.Scatter( x=yplot_2.index, y=yplot_2, name=estacion_2 ) )
    
    #finalise plot
    fig.update_layout( hovermode='x', xaxis_tickangle=-45 )
    fig.show()
    
#display interactive widget
idisp.display( widg.interactive( plot_audience_hour, estacion_1=est_1_widg, estacion_2=est_2_widg,
                                 dias=dias_widg, mes=mes_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Estación 1:', index=48, options=('CDMX XEABC-AM - (760 AM) ABC Rad…

In [12]:
html( '##', 'Dashboard Cruce' )

## <span style="color: rgb(0,71,187)">Dashboard Cruce</span>

In [13]:
#plot comparison of audience per spot for stations across time

key = 'join' #dataset key = auditsa, inra, join

#default values
comp_1_def = 'MVS Noticias'
comp_2_def = 'Exa'
sector_def = 'TELECOMUNICACIONES'
target_def = 'Total Población'

#define widgets

#radio group 1
comp_1_widg = widg.Dropdown( options=data[key]['competencia'].sort_values().unique(),
                            value=comp_1_def,
                            description='Grupo 1:' )
#radio group 2
comp_2_widg = widg.Dropdown( options=data[key]['competencia'].sort_values().unique(),
                            value=comp_2_def,
                            description='Grupo 2:' )
#sector
sector_widg = widg.Dropdown( options=data[key]['sector'].sort_values().dropna().unique(),
                             value=sector_def,
                             description='Sector:' )
#target
target_widg = widg.RadioButtons( options=data[key]['target'].sort_values().unique(),
                                 value=target_def,
                                 description='Target:' )

#plot comparison of audience per spot for stations across time
def plot_audSpot_2stationss_time( comp_1, comp_2, sector, target ):

    cond_1 = ( ( data[key]['competencia'] == comp_1 )
             & ( data[key]['sector'] == sector )
             & ( data[key]['target'] == target )
             ) #condition to filter data
    yplot_1_data = data[key][ cond_1 ].groupby( 'mes' )[[ 'hits', 'audiencia' ]].agg( np.sum ).reindex( months )
    yplot_1 = yplot_1_data[ 'audiencia' ] / yplot_1_data[ 'hits' ] #avg audience per spot

    cond_2 = ( ( data[key]['competencia'] == comp_2 )
             & ( data[key]['sector'] == sector )
             & ( data[key]['target'] == target )
             ) #condition to filter data
    yplot_2_data = data[key][ cond_2 ].groupby( 'mes' )[[ 'hits', 'audiencia' ]].agg( np.sum ).reindex( months )
    yplot_2 = yplot_2_data[ 'audiencia' ] / yplot_2_data[ 'hits' ] #avg audience per spot
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Audiencia promedio por spot en el tiempo',
                       xaxis_title='Mes',
                       yaxis_title='Audiencia promedio por spot' )
    
    #plot plot!
    fig.add_trace( go.Scatter( x=yplot_1.index, y=yplot_1, name=comp_1 ) )
    fig.add_trace( go.Scatter( x=yplot_2.index, y=yplot_2, name=comp_2 ) )
    
    #finalise plot
    fig.update_layout( hovermode='x' )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_audSpot_2stationss_time, comp_1=comp_1_widg, comp_2=comp_2_widg,
                                 sector=sector_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Grupo 1:', index=2, options=('Exa', 'La Mejor FM', 'MVS Noticias',…

In [14]:
#plot spots-audience correlation for stations in groups

key = 'join' #dataset key = auditsa, inra, join

#default values
company_def = 'Gobierno de México'
mes_def = months
plaza_def = data[key]['plaza'].sort_values().unique().tolist()
estudio_def = 'Mediómetro'
target_def = 'Total Población'

#company
company_widg = widg.Dropdown( options=data[key]['compania'].sort_values().unique(),
                               value=company_def,
                               description='Compañía:' )
#month
mes_widg = widg.SelectMultiple( options=mes_def, value=mes_def, description='Mes:' )
#place
plaza_widg = widg.SelectMultiple( options=plaza_def, value=plaza_def, description='Plaza:' )
#estudio
estudio_widg = widg.Dropdown( options=data[key]['estudio'].sort_values().unique(),
                             value=estudio_def,
                             description='Estudio:' )
#target
target_widg = widg.RadioButtons( options=data[key]['target'].sort_values().unique(),
                                 value=target_def,
                                 description='Target:' )
    
#plot spots-audience correlation for stations in groups
def plot_audSpot_corr_stations( company, mes, plaza, estudio, target ):

    cond = ( ( data[key]['compania'] == company )
           & ( data[key]['mes'].isin( mes ) )
           & ( data[key]['plaza'].isin( plaza ) )
           & ( data[key]['estudio'] == estudio )
           & ( data[key]['target'] == target )
           ) #condition to filter data
    yplot = data[key][ cond ].groupby( 'estacion' )[[ 'hits', 'audiencia', 'competencia' ]].agg({
                                     'hits' : np.sum, 'audiencia' : np.sum, 'competencia' : 'first' })    
    
    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Correlación spots-audiencia en estaciones por grupo competencia',
                       xaxis_title='Número de spots',
                       yaxis_title='Audiencia' )
    
    #plot plot!
    for yplot_name, yplot_group in yplot.groupby( 'competencia' ): #loop through groups by competencia
        fig.add_trace( go.Scatter( x=yplot_group.hits, y=yplot_group.audiencia, text=yplot_group.index,
                                   name=yplot_name, mode='markers', marker_size=plot_props['ms'],
                                   opacity=0.8 ) )
    
    #finalise plot
    fig.update_layout( xaxis_showspikes=True, yaxis_showspikes=True  )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_audSpot_corr_stations, company=company_widg,
                                 mes=mes_widg, plaza=plaza_widg, estudio=estudio_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Compañía:', index=950, options=('3 Hermanos', 'A.A. Alcohólicos An…

In [15]:
#plot spots-audience correlation for company across groups

key = 'join' #dataset key = auditsa, inra, join

#default values
sector_def = 'BEBIDAS'
comp_def = data[key]['competencia'].sort_values().dropna().unique().tolist()
mes_def = months
plaza_def = data[key]['plaza'].sort_values().unique().tolist()
estudio_def = 'Mediómetro'
target_def = 'Total Población'

#sector
sector_widg = widg.Dropdown( options=data[key]['sector'].sort_values().dropna().unique(),
                             value=sector_def,
                             description='Sector:' )
#group
comp_widg = widg.SelectMultiple( options=comp_def, value=comp_def, description='Grupo:' )
#month
mes_widg = widg.SelectMultiple( options=mes_def, value=mes_def, description='Mes:' )
#place
plaza_widg = widg.SelectMultiple( options=plaza_def, value=plaza_def, description='Plaza:' )
#estudio
estudio_widg = widg.Dropdown( options=data[key]['estudio'].sort_values().unique(),
                              value=estudio_def,
                              description='Estudio:' )
#target
target_widg = widg.RadioButtons( options=data[key]['target'].sort_values().unique(),
                                 value=target_def,
                                 description='Target:' )
    
#plot spots-audience correlation for company across groups
def plot_audSpot_corr_companies( sector, comp, mes, plaza, estudio, target ):

    cond = ( ( data[key]['sector'] == sector )
           & ( data[key]['competencia'].isin( comp ) )
           & ( data[key]['mes'].isin( mes ) )
           & ( data[key]['plaza'].isin( plaza ) )
           & ( data[key]['estudio'] == estudio )
           & ( data[key]['target'] == target )
           ) #condition to filter data
    yplot = data[key][ cond ].groupby([ 'compania', 'competencia' ], as_index=False)[[ 'hits', 'audiencia'
                                                                                    ]].agg( np.sum )

    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='Correlación spots-audiencia en compañías por grupo competencia',
                       xaxis_title='Número de spots',
                       yaxis_title='Audiencia' )
    
    #plot plot!
    for yplot_name, yplot_group in yplot.groupby( 'competencia' ): #loop through groups by competencia
        fig.add_trace( go.Scatter( x=yplot_group.hits, y=yplot_group.audiencia, text=yplot_group.compania,
                                   name=yplot_name, mode='markers', marker_size=plot_props['ms'],
                                   opacity=0.8 ) )
    
    #finalise plot
    fig.update_layout( xaxis_type='log', xaxis_showspikes=True, yaxis_type='log', yaxis_showspikes=True  )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_audSpot_corr_companies, sector=sector_widg, comp=comp_widg,
                                 mes=mes_widg, plaza=plaza_widg, estudio=estudio_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Sector:', index=4, options=('AGROPECUARIA/ANIMALES DOMESTICOS', 'A…

In [16]:
#plot ROI measure in map for company

key = 'join' #dataset key = auditsa, inra, join

#set up place coordinates
plazas = data[key]['plaza'].sort_values().unique()
plaza_coords = pd.DataFrame( np.zeros( ( len(plazas), 2 ) ), index=pd.Series( plazas, name='plaza'),
                             columns=pd.Series( [ 'lat', 'lon' ], name='coords' ) )
plaza_coords.loc[ 'Cd Juárez', : ] = 31.6904, -106.4245
plaza_coords.loc[ 'Ciudad de México', : ] = 19.4326, -99.1332
plaza_coords.loc[ 'Cuernavaca', : ] = 18.9242, -99.2216
plaza_coords.loc[ 'Guadalajara', : ] = 20.6597, -103.3496
plaza_coords.loc[ 'Monterrey', : ] = 25.6866, -100.3161
plaza_coords.loc[ 'Puebla', : ] = 19.0414, -98.2063
#plaza_coords.loc[ 'Zapopan (GDL)', : ] = 20.6720, -103.4165

colors = ['rgb(27,158,119)','rgb(217,95,2)','rgb(117,112,179)','rgb(231,41,138)','rgb(102,166,30)','rgb(230,171,2)']

#default values
company_def = 'Coca-Cola'
comp_def = data[key]['competencia'].sort_values().dropna().unique().tolist()
mes_def = months
estudio_def = 'Mediómetro'
target_def = 'Total Población'

#company
company_widg = widg.Dropdown( options=data[key]['compania'].sort_values().unique(),
                               value=company_def,
                               description='Compañía:' )
#group
comp_widg = widg.SelectMultiple( options=comp_def, value=comp_def, description='Grupo:' )
#month
mes_widg = widg.SelectMultiple( options=mes_def, value=mes_def, description='Mes:' )
#estudio
estudio_widg = widg.Dropdown( options=data[key]['estudio'].sort_values().unique(),
                              value=estudio_def,
                              description='Estudio:' )
#target
target_widg = widg.RadioButtons( options=data[key]['target'].sort_values().unique(),
                                 value=target_def,
                                 description='Target:' )
  
#plot ROI measure in map for company
def plot_map_company( company, comp, mes, estudio, target ):

    cond = ( ( data[key]['compania'] == company )
           & ( data[key]['competencia'].isin( comp ) )
           & ( data[key]['mes'].isin( mes ) )
           & ( data[key]['estudio'] == estudio )
           & ( data[key]['target'] == target )
           ) #condition to filter data
    yplot_data = data[key][ cond ].groupby( 'plaza' )[[ 'hits', 'audiencia' ]].agg( np.sum )
    plaza_coords['yplot'] = yplot_data[ 'audiencia' ] / yplot_data[ 'hits' ]
    plaza_coords.fillna( 0, inplace=True )

    plaza_coords['text'] = ''
    for plaza in plazas:
        plaza_coords.loc[ plaza, 'text' ] = plaza+'<br>ROI: {:.2f}'.format( plaza_coords.loc[plaza]['yplot'] )

    #initialise plot
    fig = go.Figure()
    fig.update_layout( title='ROI (audiencia promedio por spot) para compañía en plazas' )

    #plot plot!
    fig.add_trace(
        go.Scattergeo(
            lat=plaza_coords.lat, lon=plaza_coords.lon, text=plaza_coords.text,
            marker=dict(
                size=plaza_coords.yplot,
                sizemode='area',
                sizeref=2.*max( plaza_coords.yplot )/(50.**2),
                sizemin=4,
                color=colors
            )
        )
    )

    #finalise plot
    fig.update_layout(
        geo = dict(
            lataxis_range=[ 14, 33 ],
            lonaxis_range=[ -118, -86 ],
            showcountries=True,
            projection_type='natural earth'
        )
    )
    fig.show()

#display interactive widget
idisp.display( widg.interactive( plot_map_company, sector=sector_widg, company=company_widg, comp=comp_widg,
                                 mes=mes_widg, estudio=estudio_widg, target=target_widg ) )


interactive(children=(Dropdown(description='Compañía:', index=432, options=('3 Hermanos', 'A.A. Alcohólicos An…

In [17]:
html( '##', 'Información adicional' )

## <span style="color: rgb(0,71,187)">Información adicional</span>

In [18]:
html( '###', 'Columnas en datos' )

### <span style="color: rgb(0,71,187)">Columnas en datos</span>

In [19]:
#get columns in dataset

base_def = 'auditsa'

base_widg = widg.Dropdown( options=list(data), value=base_def, description='Base:' )

def columns_dataset( base ):
    for col in data[ base ].columns:
        print(col)
        
idisp.display( widg.interactive( columns_dataset, base=base_widg ) ) #get columns in dataset


interactive(children=(Dropdown(description='Base:', options=('auditsa', 'inra', 'join'), value='auditsa'), Out…

In [20]:
html( '###', 'Elementos más comunes' )

### <span style="color: rgb(0,71,187)">Elementos más comunes</span>

In [21]:
#get top elements in column

html( '####', 'Auditsa' )

key = 'auditsa'
top_def = 30

col_widg = widg.Dropdown( options=data[key].columns, value=data[key].columns[0], description='Columna:' )
top_widg = widg.IntText( value=top_def, description='Top:' )

def top_elements_column( col, top ):
    print( data[key][ col ].value_counts()[:top] )
    
idisp.display( widg.interactive( top_elements_column, col=col_widg, top=top_widg ) ) #get top elements in column


#### <span style="color: rgb(0,71,187)">Auditsa</span>

interactive(children=(Dropdown(description='Columna:', options=('categoria', 'compania', 'competencia', 'estac…

In [22]:
#get top elements in column

html( '####', 'Inra' )

key = 'inra'
top_def = 30

col_widg = widg.Dropdown( options=data[key].columns, value=data[key].columns[0], description='Columna:' )
top_widg = widg.IntText( value=top_def, description='Top:' )

def top_elements_column( col, top ):
    print( data[key][ col ].value_counts()[:top] )
    
idisp.display( widg.interactive( top_elements_column, col=col_widg, top=top_widg ) ) #get top elements in column


#### <span style="color: rgb(0,71,187)">Inra</span>

interactive(children=(Dropdown(description='Columna:', options=('plaza', 'mes', 'estudio', 'dias', 'target', '…

In [23]:
#get top elements in column

html( '####', 'Cruce' )

key = 'join'
top_def = 30

col_widg = widg.Dropdown( options=data[key].columns, value=data[key].columns[0], description='Columna:' )
top_widg = widg.IntText( value=top_def, description='Top:' )

def top_elements_column( col, top ):
    print( data[key][ col ].value_counts()[:top] )
    
idisp.display( widg.interactive( top_elements_column, col=col_widg, top=top_widg ) ) #get top elements in column


#### <span style="color: rgb(0,71,187)">Cruce</span>

interactive(children=(Dropdown(description='Columna:', options=('mes', 'plaza', 'estacion', 'grupo', 'competen…

In [24]:
html( '###', 'Estaciones por plaza' )

### <span style="color: rgb(0,71,187)">Estaciones por plaza</span>

In [25]:
#get stations per place

html( '####', 'Auditsa' )

key = 'auditsa'
plaza_def = 'Ciudad de México'

plaza_widg = widg.Dropdown( options=data[key]['plaza'].sort_values().unique(), value=plaza_def, description='Plaza:' )

def stations_place( plaza ):
    group_plaza = data[key].groupby( 'plaza' ).get_group( plaza )
    
    for estacion in group_plaza['estacion'].sort_values().unique():
        print( estacion )
        
idisp.display( widg.interactive( stations_place, plaza=plaza_widg ) ) #get stations per place


#### <span style="color: rgb(0,71,187)">Auditsa</span>

interactive(children=(Dropdown(description='Plaza:', index=1, options=('Cd Juárez', 'Ciudad de México', 'Cuern…

In [26]:
#get stations per place

html( '####', 'Inra' )

key = 'inra'
plaza_def = 'Ciudad de México'

plaza_widg = widg.Dropdown( options=data[key]['plaza'].sort_values().unique(), value=plaza_def, description='Plaza:' )

def stations_place( plaza ):
    group_plaza = data[key].groupby( 'plaza' ).get_group( plaza )
    
    for estacion in group_plaza['estacion'].sort_values().unique():
        print( estacion )
        
idisp.display( widg.interactive( stations_place, plaza=plaza_widg ) ) #get stations per place


#### <span style="color: rgb(0,71,187)">Inra</span>

interactive(children=(Dropdown(description='Plaza:', index=1, options=('Cd Juárez', 'Ciudad de México', 'Cuern…

In [27]:
#get stations per place

html( '####', 'Cruce' )

key = 'join'
plaza_def = 'Ciudad de México'

plaza_widg = widg.Dropdown( options=data[key]['plaza'].sort_values().unique(), value=plaza_def, description='Plaza:' )

def stations_place( plaza ):
    group_plaza = data[key].groupby( 'plaza' ).get_group( plaza )
    
    for estacion in group_plaza['estacion'].sort_values().unique():
        print( estacion )
        
idisp.display( widg.interactive( stations_place, plaza=plaza_widg ) ) #get stations per place


#### <span style="color: rgb(0,71,187)">Cruce</span>

interactive(children=(Dropdown(description='Plaza:', index=1, options=('Cd Juárez', 'Ciudad de México', 'Cuern…